In [1]:
# Mount Google Drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch torchvision segmentation-models-pytorch albumentations tqdm opencv-python scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unin

In [3]:
# Unzip the dataset
!unzip "/content/drive/MyDrive/Datasets_Assessment.zip" -d "/content/data"

Archive:  /content/drive/MyDrive/Datasets_Assessment.zip
   creating: /content/data/Datasets/
   creating: /content/data/Datasets/Raw_Images/
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_001_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_002_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_003_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_004_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_005_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_006_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_007_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_008_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_009_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_010_Set1.tiff  
  inflating: /content/data/Datasets/Raw_Images/Raw_Observation_011_S

In [4]:
!find /content/data -maxdepth 2 | sed 's/^/• /'

• /content/data
• /content/data/Datasets
• /content/data/Datasets/Raw_Images
• /content/data/Datasets/Reference_Images


In [5]:
!pip install --upgrade albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 2.0.6
    Uninstalling albumentations-2.0.6:
      Successfully uninstalled albumentations-2.0.6


In [10]:
import os, cv2, shutil
import numpy as np
from albumentations import (
    Compose,
    HorizontalFlip, VerticalFlip, RandomRotate90,
    RandomBrightnessContrast, GaussNoise, MotionBlur, ElasticTransform
)
from sklearn.model_selection import train_test_split

# ── PATH CONFIG ────────────────────────────────────────────────────────────
input_dir     = "/content/data/Datasets/Raw_Images"
aug_root      = "/content/data/Datasets/augmented"
train_out     = os.path.join(aug_root, "train")
test_out      = os.path.join(aug_root, "test")

# augmentation params
valid_exts = (".png", ".jpg", ".jpeg", ".tif", ".tiff")
n_augment    = 8
test_frac    = 0.2
seed         = 42
# ──────────────────────────────────────────────────────────────────────────

# make sure your output dirs exist
os.makedirs(train_out, exist_ok=True)
os.makedirs(test_out,  exist_ok=True)

# collect images recursively
all_images = []
for root, _, files in os.walk(input_dir):
    for fname in files:
        if fname.lower().endswith(valid_exts):
            all_images.append(os.path.join(root, fname))

if not all_images:
    raise FileNotFoundError(
        f"No images found under {input_dir!r} with extensions {valid_exts}."
    )

if not all_images:
    raise FileNotFoundError(
        f"No images found under {input_dir!r}.\n"
        "Are your files truly in there? (You may need to unzip or mount your drive.)"
    )

# Derive labels
labels = ["streak" if "streak" in os.path.basename(p) else "star"
          for p in all_images]

# Split BEFORE augment
train_paths, test_paths, train_lbls, _ = train_test_split(
    all_images, labels,
    test_size=test_frac,
    stratify=labels,
    random_state=seed
)

# Build augmentation pipeline
transform = Compose([
    RandomRotate90(),
    HorizontalFlip(),
    VerticalFlip(),
    RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
    GaussNoise(),
    MotionBlur(blur_limit=7),
    ElasticTransform(),
])

# helper: create dest path mirroring subfolders
def mirror_path(src_path, src_root, dst_root):
    rel = os.path.relpath(src_path, src_root)
    dst_dir = os.path.join(dst_root, os.path.dirname(rel))
    os.makedirs(dst_dir, exist_ok=True)
    return os.path.join(dst_dir, os.path.basename(src_path))

#  Process TRAIN set (copy + augment)
for src in train_paths:
    img = cv2.imread(src, cv2.IMREAD_UNCHANGED)
    if img is None:
        print(f"⚠️  Skipping unreadable file: {src}")
        continue
    img = img.astype(np.uint8, copy=False)

    # save original
    dst_orig = mirror_path(src, input_dir, train_out)
    cv2.imwrite(dst_orig, img)

    # save N augmented copies
    base, ext = os.path.splitext(os.path.basename(src))
    for i in range(n_augment):
        aug_img = transform(image=img)["image"]
        aug_img = np.clip(aug_img, 0, 255).astype(np.uint8)
        aug_name = f"{base}_aug{i}{ext}"
        dst_aug  = os.path.join(os.path.dirname(dst_orig), aug_name)
        cv2.imwrite(dst_aug, aug_img)

# Process TEST set (copy originals only)
for src in test_paths:
    dst = mirror_path(src, input_dir, test_out)
    shutil.copy(src, dst)

print("✅ Augmentation complete!")
print(f"  Train folder: {train_out}")
print(f"  Test  folder: {test_out}")


✅ Augmentation complete!
  Train folder: /content/data/Datasets/augmented/train
  Test  folder: /content/data/Datasets/augmented/test


In [11]:
# Train/Test Index File Writer (Fixed)

import glob

# Paths to the folders after split + augmentation
train_files = glob.glob("/content/data/Datasets/augmented/train/*.png")
test_files  = glob.glob("/content/data/Datasets/augmented/test/*.png")

# Write train.txt
with open("train.txt", "w") as f:
    for img in train_files:
        lbl = "streak" if "streak" in img else "star"
        f.write(f"{img} {lbl}\n")

# Write test.txt
with open("test.txt", "w") as f:
    for img in test_files:
        lbl = "streak" if "streak" in img else "star"
        f.write(f"{img} {lbl}\n")


In [12]:
# Model Definition & Training

import torch
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=2
).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [13]:
import torch
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=2
).to(device)

In [16]:
import os, glob, random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn

# ─── CONFIG ───────────────────────────────────────────────────────────────
raw_dir    = "/content/data/Datasets/Raw_Images"
train_idx  = "/content/train.txt"
test_idx   = "/content/test.txt"
extensions = (".png", ".jpg", ".jpeg", ".tif", ".tiff")

# fraction for test-split
test_frac   = 0.2
random_seed = 42

# hyperparams
batch_size  = 2
num_workers = 4
num_epochs  = 10
lr          = 1e-4
wd          = 1e-5
# ──────────────────────────────────────────────────────────────────────────

# 1) Auto-generate train/test index files if they don't exist or are empty
def build_index_files():
    # find all images under raw_dir
    all_imgs = [
        p for p in glob.glob(f"{raw_dir}/**/*", recursive=True)
        if p.lower().endswith(extensions)
    ]
    if not all_imgs:
        raise FileNotFoundError(f"No images found under {raw_dir!r}")

    random.seed(random_seed)
    random.shuffle(all_imgs)
    split = int(len(all_imgs) * (1 - test_frac))
    train_list = all_imgs[:split]
    test_list  = all_imgs[split:]

    # write them out
    with open(train_idx, "w") as f:
        for p in train_list:
            label = "streak" if "streak" in os.path.basename(p) else "star"
            f.write(f"{p} {label}\n")

    with open(test_idx, "w") as f:
        for p in test_list:
            label = "streak" if "streak" in os.path.basename(p) else "star"
            f.write(f"{p} {label}\n")

# regenerate if needed
if (not os.path.isfile(train_idx)) or os.path.getsize(train_idx) == 0:
    print("🔧 Building fresh train/test index files...")
    build_index_files()

# ─── Dataset & Dataloaders ────────────────────────────────────────────────
class StreakStarDataset(Dataset):
    def __init__(self, index_file, transform=None):
        with open(index_file, "r") as f:
            lines = f.read().splitlines()
        self.samples   = [line.split(maxsplit=1) for line in lines]
        self.transform = transform
        self.class_map = {"streak": 0, "star": 1}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label_str = self.samples[idx]
        img   = Image.open(img_path).convert("L")
        if self.transform:
            img = self.transform(img)
        label = self.class_map[label_str]
        return img, label

# transforms
transform = transforms.Compose([
    transforms.Resize((4418, 4418)),
    transforms.ToTensor(),
])

# datasets
train_ds = StreakStarDataset(train_idx, transform=transform)
test_ds  = StreakStarDataset(test_idx,  transform=transform)

# loaders
train_loader = DataLoader(train_ds, batch_size=batch_size,
                          shuffle=True,  num_workers=num_workers)
test_loader  = DataLoader(test_ds,  batch_size=batch_size,
                          shuffle=False, num_workers=num_workers)

# ─── Model, Loss, Optimizer ───────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet34(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc    = nn.Linear(model.fc.in_features, 2)
model       = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

# ─── Training Loop ────────────────────────────────────────────────────────
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0

    for imgs, labels in train_loader:
        imgs   = imgs.to(device)
        labels = labels.to(device)

        logits = model(imgs)
        loss   = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)

    avg_loss = running_loss / len(train_ds)
    print(f"Epoch {epoch:02d}  Train Loss: {avg_loss:.4f}")

print("✅ Training complete.")


🔧 Building fresh train/test index files...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 223MB/s]


Epoch 01  Train Loss: 0.1595
Epoch 02  Train Loss: 0.0095
Epoch 03  Train Loss: 0.0024
Epoch 04  Train Loss: 0.0013
Epoch 05  Train Loss: 0.0010
Epoch 06  Train Loss: 0.0008
Epoch 07  Train Loss: 0.0007
Epoch 08  Train Loss: 0.0006
Epoch 09  Train Loss: 0.0006
Epoch 10  Train Loss: 0.0005
✅ Training complete.


In [17]:

test_ds = StreakStarDataset("/content/test.txt", transform=transform)
test_loader = DataLoader(
    test_ds,
    batch_size=8,
    shuffle=False,
    num_workers=4
)



In [18]:
# Evaluation & Metrics

from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

model.eval()  # set to eval mode

all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.cpu().numpy()

        logits = model(images)
        preds  = logits.argmax(dim=1).cpu().numpy()

        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# Compute metrics (default: pos_label=1 → 'star')
precision = precision_score(all_labels, all_preds, pos_label=1)
recall    = recall_score(all_labels, all_preds, pos_label=1)
f1        = f1_score(all_labels, all_preds, pos_label=1)
iou       = jaccard_score(all_labels, all_preds, pos_label=1)

print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, "
      f"F1: {f1:.3f}, IoU: {iou:.3f}")


Precision: 1.000, Recall: 1.000, F1: 1.000, IoU: 1.000
